<a href="https://colab.research.google.com/github/bilgin-kocak/fenics-tutorials-anders-logg/blob/master/Exercise_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise 3: Solving Nonlinear Problems
![image.png](attachment:image.png)

In this exercise, we will solve the following nonlinear PDE:

\begin{alignat}{2}
-\nabla\cdot(a(u)\nabla u) &= f(u) \quad && \text{ in } \Omega \\
u &= u_{\mathrm{L}} \quad && \text{ on } \Gamma_{\mathrm{L}} \\
u &= u_{\mathrm{R}} \quad && \text{ on } \Gamma_{\mathrm{R}} \\
-a(u)\partial_n u &= g \quad && \text{ on } \Gamma_{\mathrm{N}}
\end{alignat}

We will use the following data:

* $\Omega = (0,1)^3$ (the unit cube)
* $a(u) = 1 + u^2$
* $f(u) = u^3$
* $\Gamma_{\mathrm{L}} = \{(x,y,z) \in \partial\Omega \mid x = 0\}$ (the left boundary)
* $\Gamma_{\mathrm{R}} = \{(x,y,z) \in \partial\Omega \mid x = 1\}$ (the right boundary)
* $\Gamma_{\mathrm{N}} = \partial\Omega \setminus (\Gamma_{\mathrm{L}} \cup \Gamma_{\mathrm{R}})$
* $u_{\mathrm{L}} = \sin(2\pi yz)$
* $u_{\mathrm{R}} = \sin(2\pi (1-yz))$
* $g = 0$

For hints and more reading on how to solve this problem in FEniCS, see the [section on the nonlinear Poisson problem](https://fenicsproject.org/pub/tutorial/html/._ftut1007.html#ftut1:gallery:nonlinearpoisson) in the [FEniCS Tutorial](https://fenicsproject.org/tutorial/).

## Importing FEniCS
The first thing we need to do is to import FEniCS.

In [ ]:
# Installing and Importing Fenics
from google.colab import files

import platform, sys
python_version=platform.python_version()
from distutils.version import LooseVersion, StrictVersion

if ( LooseVersion(python_version) < LooseVersion("3.0.0")):
    print("Python3 is needed!");
    print("How to fix: Runtime/Change_runtime_type/Python 3");
    sys.exit()
try:
    from fenics import *; from mshr import *
except ImportError as e:
    !apt-get install -y -qq software-properties-common python-software-properties module-init-tools
    !add-apt-repository -y ppa:fenics-packages/fenics
    !apt-get update -qq
    !apt install -y --no-install-recommends fenics
    from fenics import *; from mshr import *
    
import matplotlib.pyplot as plt;
from IPython.display import clear_output, display; import time; import dolfin.common.plotting as fenicsplot 
import time

import os, sys, shutil

dolfin_version = dolfin.__version__
print ('dolfin version:', dolfin_version)

!rm -rf * # clean up all files
# Useful commands
# Remove an empty folder      : os.rmdir("my_results")
# Remove a folder with files  : shutil.rmtree("results")
# Make a folder               : os.mkdir("my_results")
# Runtime/Change_runtime_type/Python3

E: Package 'python-software-properties' has no installation candidate
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://ppa.launchpad.net/fenics-packages/fenics/ubuntu bionic InRelease [15.4 kB]
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/pp

## Initializing notebook plotting

When running in a Jupyter notebook, a special line is necessary to show FEniCS plots in the notebook.

In [ ]:
%matplotlib inline

## Creating the mesh
Create the mesh by using the built-in class `UnitCubeMesh`. A mesh size of $h=1/24$ is reasonable for this exercise.

In [ ]:
mesh = UnitCubeMesh(24,24,24)
mesh

## Defining the finite element function space
When we have created the mesh, we can define the finite element function space. We will use a standard piecewise linear $P_1$ function space.

In [ ]:
V = FunctionSpace(mesh, 'P', 1)

Calling FFC just-in-time (JIT) compiler, this may take some time.


## Defining the boundary condition
The boundary conditions can be defined by specifying all points on the boundary close to the left and right boundaries. Use an `Expression` to define $u_\mathrm{L}$ and $u_\mathrm{R}$, and the `DirichletBC` class to define the two boundary conditions. Collect both boundary conditions in a variable named `bcs`.

In [ ]:
def leftBC(x, on_boundary):
    return on_boundary and near(x[0],0)
def rightBC(x, on_boundary):
    return on_boundary and near(x[0],1)
u_L = Expression('sin(2*pi*x[1]*x[2])',degree=2)
bc_L = DirichletBC(V, u_L, leftBC)
u_R = Expression('sin(2*pi*(1 - x[1]*x[2]))',degree=2)
bc_R = DirichletBC(V, u_R, leftBC)
bcs = [bc_L, bc_R]

## Defining the variational problem
The finite element variational problem for the nonlinear PDE reads: Find $u \in V$ such that
$$
\int_{\Omega} (1 + u^2) \, \nabla u \cdot \nabla v \,\mathrm{d}x
= \int_{\Omega} u^3 \, v \,\mathrm{d}x
$$
for all $v \in V$.

This can be expressed in FEniCS by defining the nonlinear form $F$:

\begin{align}
  F(u; v) = \int_{\Omega} (1 + u^2) \, \nabla u \cdot \nabla v \,\mathrm{d}x 
  - \int_{\Omega} u^3 \, v \,\mathrm{d}x
\end{align}
and solving $F(u; v) = 0$ for all $v \in V$.

(Note that $F$ is technically a *linear* form in FEniCS since it is linear in the test function $v$ and is assembled into a vector, in contrast to a *bilinear* form which is linear in both $u$ and $v$ and assembled into a matrix.)

In [ ]:
u = Function(V)
v = TestFunction(V)
F = (1+u**2)*dot(grad(u),grad(v))*dx - u**3*v*dx

## Solving the variational problem
The variational problem can now be solved by calling the FEniCS `solve` command. Use the `plot` command to visualize the solution. 

In [ ]:
solve(F==0,u, bcs)
# plot(u)
u.vector()[:]

array([-2.44929360e-16, -2.44929360e-16, -3.99698601e-01, ...,
       -4.52588235e-01, -3.26968011e-01, -2.44929360e-16])

## Postprocessing
To postprocess the solution using an external postprocessor, for example Paraview, save the solution to VTK format (file suffix `.pvd`).